In [1]:
import torchaudio
import torch
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from ast import literal_eval

from helpers.AudioVisionDataset import AudioVisionDataset
from helpers.ModelLayout import AudioVisionModel
from torchsummary import summary

In [2]:
features_file = "../DataAnalysis/8KFeatures_new.csv"

In [3]:
featuresdf = pd.read_csv(features_file)
temp = literal_eval(featuresdf['feature'][0])
print(temp)

((-306.27045, 82.50227, -13.483826, 39.512886, 10.829396, -5.5336895, 21.208496, 14.331081, -1.2811562, -0.114278086, 5.1434336, 9.1233225, -18.58668, -7.0402246, 4.9211473, -7.1048756, -3.0341606, 8.068735, -4.4463882, -11.155183, -5.2065735, 6.5561895, -4.356899, -5.742895, -7.4755507, -9.127256, -12.907261, -9.089582, -1.4352409, -3.518753, -4.469093, 1.9015925, 5.9759874, -1.4129353, -3.7519631, 2.471726, 1.742979, -1.9519746, -11.749705, -2.2869136), (-300.84735, 92.29649, -13.909019, 39.164227, 5.844798, -7.447727, 15.783237, 14.292462, -5.2948456, -0.56967425, 0.46488976, 9.3216505, -12.641883, -6.363125, 4.015251, -3.4300652, -5.5678263, 9.736972, -0.15720987, -8.997645, -9.722063, 3.7748594, -6.2298555, -8.30838, -5.182138, -5.9952707, -15.246671, -2.2106001, 3.7390344, 2.7662132, -5.1053886, 0.5738433, 0.64037025, -1.5612862, -2.6556425, 0.69916356, 0.90216947, -3.12309, -9.492268, -0.85865915), (-299.80707, 94.57649, -12.958305, 40.39865, 0.35747427, 2.8329403, 19.129028, 16

In [4]:
print(len(temp))
print(len(temp[0]))

109
40


In [5]:
# featuresdf = pd.read_csv(features_file)
# temp = literal_eval(featuresdf['feature'][0])
# print(temp)
# print(len(temp))

In [6]:
av_dataset = AudioVisionDataset(features_file)

In [7]:
train_size = int(0.667 * len(av_dataset))
test_size = len(av_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(av_dataset, [train_size, test_size])

In [8]:
model = AudioVisionModel()

In [9]:
import torch.utils.data as data
train_data_loader = data.DataLoader(train_dataset, shuffle = True)
test_data_loader = data.DataLoader(test_dataset, shuffle = True)

In [10]:
summary(model, (1,40, 109))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 14, 39, 108]              70
         MaxPool2d-2           [-1, 14, 19, 54]               0
           Dropout-3           [-1, 14, 19, 54]               0
            Conv2d-4           [-1, 28, 18, 53]           1,596
         MaxPool2d-5            [-1, 28, 9, 26]               0
           Dropout-6            [-1, 28, 9, 26]               0
            Conv2d-7            [-1, 42, 8, 25]           4,746
         MaxPool2d-8            [-1, 42, 4, 12]               0
           Dropout-9            [-1, 42, 4, 12]               0
           Linear-10                   [-1, 64]         129,088
          Dropout-11                   [-1, 64]               0
           Linear-12                   [-1, 10]             650
Total params: 136,150
Trainable params: 136,150
Non-trainable params: 0
-------------------------------

In [11]:
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.7)

In [12]:
import time

def trainNet(net, n_epochs):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", 174)
    print("epochs=", n_epochs)
    print("learning_rate=", 0.001)
    print("=" * 30)
    
    n_batches = len(train_data_loader)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        count = 0
        
        for i, data in enumerate(train_data_loader, 0):
            count+= 1
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs.unsqueeze(0)), Variable(labels)

            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in test_data_loader:
            
            count += 1
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs.unsqueeze(0)), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(test_data_loader)))
        print(count)
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [ ]:
trainNet(model, 2)

===== HYPERPARAMETERS =====
batch_size= 174
epochs= 2
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 2.34 took: 13.45s
Epoch 1, 20% 	 train_loss: 2.29 took: 13.76s
Epoch 1, 30% 	 train_loss: 2.27 took: 13.61s
Epoch 1, 40% 	 train_loss: 2.28 took: 13.53s
Epoch 1, 50% 	 train_loss: 2.28 took: 13.57s
Epoch 1, 60% 	 train_loss: 2.26 took: 14.21s
